In [30]:
import fitz
fileName = "julio 2025"
year = "2025"
op = "AccountNu"
pdfFileName = rf"Data\pdfFiles\{op}\{fileName}.pdf"
htmlFileName = rf"Data\htmlFiles\{op}\{fileName}.html"
doc = fitz.open(pdfFileName)
html = ''

for num_pagina in range(1,len(doc)):
    pagina = doc.load_page(num_pagina)
    html += pagina.get_text("html")

with open(htmlFileName, 'w', encoding="utf-8") as f:
    f.write(html)


In [31]:
def to_num(x: str):
    return float(x.replace('$','').replace(',','').replace(' ',''))

def is_date(x:str):
    temp = x.split(' ')
    months = set([
        "ENE", "FEB", "MAR", "ABR", "MAY", "JUN",
        "JUL", "AGO", "SEP", "OCT", "NOV", "DIC"
    ])
    try:
        int(x[0])
    except:
        return False
    if(len(temp) < 2): return False

    return (temp[1] in months)

def to_date(x:str):
    temp = x.split(' ')
    months = {
        "ENE": "01",
        "FEB": "02",
        "MAR": "03",
        "ABR": "04",
        "MAY": "05",
        "JUN": "06",
        "JUL": "07",
        "AGO": "08",
        "SEP": "09",
        "OCT": "10",
        "NOV": "11",
        "DIC": "12"
    }
    temp[1]=months[temp[1]]
    temp.append(year)

    return temp[0]+'-'+temp[1]+'-'+temp[2]

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "lxml")
tags = soup.find_all("span")
headers = ["timestamp", "type", "description", "value"]
data = []
i = 0

while i < len(tags):
    tag = tags[i].text.strip()
    while(not is_date(tag) and tag != "Saldo final del periodo"):
        i+=1
        tag = tags[i].text.strip()
    if(tag == "Saldo final del periodo"): break
    else:
        temp = []
        for j in range(4):
            if(j == 0): temp.append(to_date(tag))
            elif(j == 3): temp.append(to_num(tag))
            else: temp.append(tag)
            i+=1
            tag = tags[i].text.strip()
        data.append(temp)



In [41]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "lxml")
tags = soup.find_all("span")
headers = ["timestamp", "description", "value"]
data = []
i = 0
last = "Con estos movimientos, tu saldo promedio del periodo fue de"

while i < len(tags):
    tag = tags[i].text.strip()
    while(not is_date(tag) and not last in tag):
        i+=1
        tag = tags[i].text.strip()
    if(last in tag): break
    temp = []
    for j in range(3):
        if(j == 0): temp.append(to_date(tag))
        elif(j == 2): temp.append(to_num(tag))
        else: temp.append(tag)
        i+=1
        tag = tags[i].text.strip()
    data.append(temp)

print(data)


[['31-07-2025', 'Retiro de Cajita: Cajita Turbo', 150.0], ['31-07-2025', 'Irving Ignacio Belmontes Diaz Transferencia', -240.0], ['31-07-2025', 'Retiro de Cajita: Cajita Turbo', 240.0], ['31-07-2025', 'Jonathan Raymundo Torres Cardenas  Transferencia', -85.0], ['31-07-2025', 'Retiro de Cajita: Cajita Turbo', 85.0], ['30-07-2025', 'Jonathan Raymundo Torres Cardenas  Transferencia', -102.0], ['30-07-2025', 'Retiro de Cajita: Cajita Turbo', 102.0], ['30-07-2025', 'Depósito en Cajita: Cajita Turbo', -900.0], ['28-07-2025', 'CHRISTOPHER ABREK GUERRERO ALFARO        Pago', 900.0], ['26-07-2025', 'abrek Transferencia', -999.0], ['26-07-2025', 'abrek Transferencia', -1.0], ['26-07-2025', 'Retiro de Cajita: Cajita Turbo', 1000.0], ['25-07-2025', 'Depósito en Cajita: Cajita Turbo', -20600.0], ['25-07-2025', 'JONATHAN RAYMUNDO TORRES CARDENAS        yo', 20600.0], ['23-07-2025', 'Depósito en Cajita: Cajita Turbo', -9.0], ['23-07-2025', 'Pago a tu tarjeta de crédito Nu', -19991.0], ['23-07-2025', 

In [45]:
import csv

with open(rf"Data\parsed\{op}\{fileName}.csv", 'w', newline='', encoding="utf-8") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(headers)
    writer.writerows(data)
